In [1]:
from scipy.optimize import curve_fit
import datetime as dt
import pandas as pd
import numpy as np
import altair as alt

data = pd.read_csv("corona_infections.csv", header=0, names=["day", "cases"])
data["day"] = [dt.datetime.strptime(d + ' 2020', "%b %d %Y") for d in data["day"]]
data

,day,cases
0,2020-02-15,16
1,2020-02-16,16
2,2020-02-17,16
3,2020-02-18,16
4,2020-02-19,16
5,2020-02-20,16
6,2020-02-21,16
7,2020-02-22,16
8,2020-02-23,16
9,2020-02-24,16


In [2]:
infections = alt.Chart(data).mark_line(point=True).encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("cases:Q", title="Corona-Fallzahlen"))
infections

alt.Chart(...)

In [3]:
cases_since_quarantine = np.array(
    data[data["day"] >= dt.datetime(year=2020, month=3, day=20)]["cases"])
day_numbers_since_quarantine = np.array(
    [d.toordinal() for d in data[data["day"] >= dt.datetime(year=2020, month=3, day=20)]["day"]])
cases_since_quarantine

array([19848, 22364, 24873, 29056])

In [4]:
def corona_curve(x, base, x_offset, y_offset):
    return pow(base=base, exp=(x - x_offset)) + y_offset
params, _ = curve_fit(corona_curve, 
          xdata=day_numbers_since_quarantine,
          ydata=cases_since_quarantine,
          p0=[1.2, dt.datetime(year=2020, month=3, day=20).toordinal(), cases_since_quarantine[0]])
params

<ipython-input-4-592d237b8822>:2: RuntimeWarning: overflow encountered in power
  return pow(base=base, exp=(x - x_offset)) + y_offset


array([1.34168374e+00, 7.37474181e+05, 1.35387481e+04])

In [5]:
projection_data = data.copy()
projection_data = projection_data.drop(["cases"], axis=1)
projection_data["curve"] = [corona_curve(d.toordinal(), *params) for d in projection_data["day"]]
for i in range(9):
    projection_data = projection_data.append(pd.DataFrame(
        [[list(projection_data["day"])[-1] + dt.timedelta(days=1), 
          corona_curve((list(projection_data["day"])[-1] + dt.timedelta(days=1)).toordinal(), *params)]],
        columns=("day", "curve")
    ), ignore_index=True)
projection_data

,day,curve
0,2020-02-15,13539.040733
1,2020-02-16,13539.140714
2,2020-02-17,13539.274858
3,2020-02-18,13539.454836
4,2020-02-19,13539.696310
5,2020-02-20,13540.020291
6,2020-02-21,13540.454972
7,2020-02-22,13541.038176
8,2020-02-23,13541.820651
9,2020-02-24,13542.870485


In [6]:
infections_chart = alt.Chart(data).mark_line(point=True, color="red").encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("cases:Q", title="Corona-Fallzahlen"))
projection_chart = alt.Chart(projection_data).mark_line(point=True).encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("curve:Q", title="Fallzahlen (Projektion)"))
infections_chart + projection_chart

alt.LayerChart(...)

In [7]:
exponent = 0.99
projection_dream = projection_data.copy()
projection_dream["curve"] = [corona_curve(d.toordinal(), exponent, params[1], params[2]) for d in projection_dream["day"]]
infections_chart = alt.Chart(data).mark_line(point=True, color="red").encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("cases:Q", title="Corona-Fallzahlen"))
projection_chart = alt.Chart(projection_data).mark_line(point=True).encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("curve:Q", title="Fallzahlen (Projektion)"))
projection_dream_chart = alt.Chart(projection_dream).mark_line(point=True, color="green").encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("curve:Q", title="Fallzahlen (Wunsch)"))
infections_chart + projection_chart + projection_dream_chart

alt.LayerChart(...)

In [8]:
bev_de = 83019213
infizierte_pro_tag = ["{:.0f}".format(bev_de*0.7/(365*dauer)) for dauer in [0.5, 1, 1.5]]
infizierte_pro_tag

['318430', '159215', '106143']